In [ ]:
# @title Upload file 'AttendeeReport.csv'
from google.colab import files
uploaded = files.upload()

In [ ]:
# @title Nhập thời gian bắt đầu, kết thúc
ngay = "2025-03-07" # @param {"type":"date"}
bat_dau = '9:00:00' # @param {"type":"string","placeholder":"9:00:00"}
ket_thuc = '10:52:00' # @param {"type":"string","placeholder":"10:00:00"}


In [ ]:
print(f"Bắt đầu: {ngay} - {bat_dau}")
print(f"Kết thúc: {ngay} - {ket_thuc}")

Bắt đầu: 2025-03-07 - 9:00:00
Kết thúc: 2025-03-07 - 10:52:00


### Code running (calculating duration)

In [ ]:
import polars as pl
from datetime import datetime, timedelta, timezone

In [ ]:
filename = 'AttendeeReport.csv'

In [ ]:
# Parse time & set timezone UTC+7
ending_datetime = datetime.combine(
    datetime.strptime(ngay, "%Y-%m-%d").date(),
    datetime.strptime(ket_thuc, "%H:%M:%S").time()
).replace(tzinfo=timezone(timedelta(hours=7)))

In [ ]:
df = pl.read_csv(filename).rename({
    'Session Id': 'id',
    'Participant Id': 'email',
    'Full Name': 'name',
    'UserAgent': 'user_agent',
    'UTC Event Timestamp': 'timestamp',
    'Action': 'action',
    'Role': 'role'
}).with_columns(
    pl.col('timestamp')
        .str.strptime(pl.Datetime, format='%m/%d/%Y %I:%M:%S %p')
        .dt.convert_time_zone('Asia/Ho_Chi_Minh')
).sort(
    by=['email','id', 'timestamp']
)

df

In [ ]:
error_entry_sessions = df.group_by('id').agg(pl.len().alias('count')).filter(pl.col('count') != 2)
df_filtered = df.join(error_entry_sessions, on='id', how='anti')

error_entry_sessions

In [ ]:
sessions = (
    df_filtered.sort(['id', 'timestamp'])
    .group_by('id')
    .agg(
        pl.col('name').first().alias('name'),
        pl.col('email').first().alias('email'),
        pl.col('timestamp').filter(pl.col('action') == 'Joined').first().alias('join_time'),
        pl.col('timestamp').filter(pl.col('action') == 'Left').first().alias('leave_time')
    )
    .with_columns(
        (pl.col('leave_time') - pl.col('join_time')).alias('duration')
    )
)

sessions

In [ ]:
def merge_intervals(intervals):
    # Sort intervals by start time
    sorted_intervals = sorted(intervals, key=lambda x: x['join_time'])

    if not sorted_intervals:
        return timedelta(seconds=0)

    merged = [sorted_intervals[0]]
    for current in sorted_intervals[1:]:
        previous = merged[-1]
        if current['join_time'] <= previous['leave_time']:
            merged[-1] = {
                'join_time': previous['join_time'],
                'leave_time': max(previous['leave_time'], current['leave_time'])
            }
        else:
            merged.append(current)

    # Calculate total duration using timedelta objects
    total_seconds = 0
    for interval in merged:
        start = interval['join_time']
        end = interval['leave_time']

        # If ending_datetime is specified and is before the end of this interval,
        # truncate the interval at ending_datetime
        if ending_datetime is not None and end > ending_datetime:
            end = ending_datetime

        # If ending_datetime is before the start of this interval, skip the whole calculation
        if ending_datetime is not None and start >= ending_datetime:
            continue

        total_seconds += (end - start).total_seconds()

    return timedelta(seconds=total_seconds)

In [ ]:
result = (
    # Group by email to process each person
    sessions
    .group_by('email')
    .agg(
        pl.col('name').first().alias('name'),

        # Output all intervals as struct array
        pl.struct(['join_time', 'leave_time']).alias('intervals'),
        pl.concat_str([
            pl.col('join_time').dt.time().cast(pl.Utf8),
            pl.lit(' - '),
            pl.col('leave_time').dt.time().cast(pl.Utf8)
        ]).alias('formatted_intervals')
    )
    .with_columns(
        pl.col('intervals').map_elements(merge_intervals, return_dtype=pl.Duration).alias('total_non_overlapping_duration'),
        pl.col('formatted_intervals')
            .list.eval(pl.format('<{}>', pl.element()))
            .list.join('')
    )
    .rename({'formatted_intervals':'formatted_intervals (UTC+7)'})
    .sort('total_non_overlapping_duration')
)

In [ ]:
result_formatted = (
    result
    .with_columns(
        (pl.col("total_non_overlapping_duration") / 1_000_000).cast(pl.Int64).alias("duration_seconds"),
    )
    .with_columns(
        (
          (pl.col("duration_seconds") // 3600).cast(pl.Utf8).str.zfill(2) + ":" +
          ((pl.col("duration_seconds") % 3600) // 60).cast(pl.Utf8).str.zfill(2) + ":" +
          (pl.col("duration_seconds") % 60).cast(pl.Utf8).str.zfill(2)
        ).alias("formatted_time")
    )
    .drop(['intervals', 'total_non_overlapping_duration'])
    .rename({
        'email': 'Email',
        'name': 'Name',
        'formatted_intervals (UTC+7)': 'Sessions (UTC+7)',
        'duration_seconds': 'Total time (seconds)',
        'formatted_time': 'Total time',
    })
)

result_formatted

In [ ]:
result_formatted.write_csv(f"Tính_TG_{ngay}.csv")